In [ ]:
import skimage
from scipy.fftpack import fft, ifft, fftshift
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny
import imutils

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from scipy import stats


Horizontal projection

In [2]:
def calcula_proj(img):
    temp=np.bitwise_not(img)
    project=np.sum(temp,1)
    return project

def pico_projection(projection):
    return np.max(projection)

def rotate(img, angle):
    temp=np.bitwise_not(img)
    rotate=imutils.rotate_bound(temp, angle)
    return np.bitwise_not(rotate)

def horizontal_projection(src):
    angulo_encontrado=0
    vp=0
    for angulo in range(180):
        aux=rotate(src,angulo)
        projecao=calcula_proj(aux)
        valor_pico=pico_projection(projecao)
        if(valor_pico>vp):
            angulo_encontrado=angulo
            vp=valor_pico
    if(angulo_encontrado<=90):
        
        return (angulo_encontrado-90)
    else:
        return -1*(180-angulo_encontrado)


Hough transform

In [3]:
def moda(array):
    #print(array)
    #counts = np.histogram(array)
    #return counts[1][np.argmax(counts[0])]
    m = stats.mode(array)
    return m[0][0]


def hough_method(image):
    minima_amostragem=5 #minima amostragem de picos
    factor=1#fator de ajuste do valor dos picos

    image = canny(image, 2, 1, 25)
    h, theta, d = hough_line(image)
    picos=[[0],[0],[0]]
    while(len(picos[0])<minima_amostragem):
        factor/=2
        picos=hough_line_peaks(h,theta,d, threshold=np.max(h)*factor)
    saved_angles=(np.rad2deg(picos[1]))
    saved_angles=np.round(saved_angles)
    moda_ang=moda(saved_angles)
    if(moda_ang<0):
        return -1*(90-abs(moda_ang))
    else:
        return 90-moda_ang
    

#angulos positivos rotacionam a imagem no sentido horario
#angulos negativos rotacionam a imagem no sentido anti-horario
# def rotate(img, angle):
#     temp=np.bitwise_not(img)
#     if(angle>= 0):
#         adjust=1
#     else:
#         adjust=-1
    
#     if(angle==0):
#         cv2.imshow("resp",temp)
#         return img
#     else:
#         if(angle <=90 and angle>=0): # angulo positivo
#             rotated = imutils.rotate_bound(temp, adjust*(90-angle))#eixo y=angulo 0, para 
#                                                                     #arrumar o angulo devemos tomar o complemento
#             rotated=np.bitwise_not(rotated)#ajusta novamente a cor da imagem
#             return (rotated,adjust*(90-angle))
#         elif(angle<0):#angulo negativo
#             rotated = imutils.rotate_bound(temp, adjust*(90-abs(angle)))#tome o complemento absoluto, e multiplique por -1
#             rotated=np.bitwise_not(rotated)#ajusta novamente a cor da imagem
#             return (rotated,adjust*(90-abs(angle)))
            

Fourier

In [4]:
def fourier_method(img):
    #calcula a transformada de fourier
    fourier = np.fft.fft2(img)
    fourier_shifted = np.fft.fftshift(fourier)
    magnitude_spectrum = 20*np.log(np.abs(fourier_shifted))
    magnitude_spectrum=np.round(magnitude_spectrum*(255/np.max(magnitude_spectrum)))
    
    #calcula o mapa de bordas do aspectro de frequencia
    border_map = canny(magnitude_spectrum, 2, 1, 25)
    h, theta, d = hough_line(border_map)
    
    #pega o angulo encontrado pela transformada de hough
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d, num_peaks=1)):
        angulo_final=np.rad2deg(angle)
    #print(angulo_final)
    if angulo_final >0:
        #print(angulo_final-90)
        angulo_ajustado=angulo_final-90
    else:
        #print(angulo_final+90)
        angulo_ajustado=angulo_final+90
        
    if(angulo_ajustado<0):
        return -1*(90-abs(angulo_ajustado))
    else:
        return 90-angulo_ajustado




#angulos positivos rotacionam a imagem no sentido horario
#angulos negativos rotacionam a imagem no sentido anti-horario
# def rotate(img, angle):
#     temp=np.bitwise_not(img)
#     if(angle>= 0):
#         adjust=1
#     else:
#         adjust=-1
        
    
#     if(angle <=90 and angle>=0): # angulo positivo
#         rotated = imutils.rotate_bound(temp, adjust*(90-angle))#eixo y=angulo 0, para 
#                                                                     #arrumar o angulo devemos tomar o complemento
#         rotated=np.bitwise_not(rotated)#ajusta novamente a cor da imagem
#         return (rotated,adjust*(90-angle))
#     else:#angulo negativo
#         rotated = imutils.rotate_bound(temp, adjust*(90-abs(angle)))#tome o complemento absoluto, e multiplique por -1
#         rotated=np.bitwise_not(rotated)#ajusta novamente a cor da imagem
#         return (rotated,adjust*(90-abs(angle)))
            

In [5]:
if(os.path.isdir("results")==False):
    os.mkdir("results")
    
if(os.path.isdir("results/all_methods")==False):
    os.mkdir("results/all_methods")
    
if(os.path.isdir("espurios")==False):
    os.mkdir("espurios")
    
if(os.path.isdir("espurios/all_methods")==False):
    os.mkdir("espurios/all_methods")
    
path="entradas/test_set1_rotated"
dest="results/fourier/test_set1_rotated_fourier"
#dest_erro="espurios/fourier/test_set1_error"

if(os.path.isdir(dest)==False):
    os.mkdir(dest)
dataframe=pd.DataFrame(columns=["imagem","Hprojection","Hough","Fourier","Esperado"])
tolerancia=15
erros=0
i=0
for root, dirs, files in os.walk(path):
    for file in files:
        if ".png" in file:
            try:
                i+=1
                img= cv2.imread(root+"/"+file,cv2.IMREAD_GRAYSCALE)
                print("calculando imagem {}".format(i))
                ang_fourier=np.round(fourier_method(img))
                ang_hough=hough_method(img)
                ang_hproj=horizontal_projection(img)
                txt=open(root+"/"+file[:-4]+"txt", "r")
                linha=txt.read()
                valor_esperado=int(linha[:-1])
                dataframe.loc[len(dataframe)]=[file,ang_hproj,ang_hough,ang_fourier,valor_esperado]
                #print([ang_hproj,ang_hough,ang_fourier,valor_esperado])
                #np.median(ang_hproj,ang_hough,ang_fourier)

                txt.close()
            except:
                continue
                
numeros=range(dataframe.shape[0])
dataframe["index_img"]=numeros
dataframe.to_csv("csv/{}_all_methods_analise.csv".format(path.split("/")[-1]))

calculando imagem 1
calculando imagem 2
calculando imagem 3
calculando imagem 4
calculando imagem 5
calculando imagem 6
calculando imagem 7
calculando imagem 8
calculando imagem 9
calculando imagem 10
calculando imagem 11
calculando imagem 12
calculando imagem 13
calculando imagem 14
calculando imagem 15
calculando imagem 16
calculando imagem 17
calculando imagem 18
calculando imagem 19
calculando imagem 20
calculando imagem 21
calculando imagem 22
calculando imagem 23
calculando imagem 24
calculando imagem 25
calculando imagem 26
calculando imagem 27
calculando imagem 28
calculando imagem 29
calculando imagem 30
calculando imagem 31
calculando imagem 32
calculando imagem 33
calculando imagem 34
calculando imagem 35
calculando imagem 36
calculando imagem 37
calculando imagem 38
calculando imagem 39
calculando imagem 40
calculando imagem 41
calculando imagem 42
calculando imagem 43
calculando imagem 44
calculando imagem 45
calculando imagem 46
calculando imagem 47
calculando imagem 48
c

calculando imagem 379
calculando imagem 380
calculando imagem 381
calculando imagem 382
calculando imagem 383
calculando imagem 384
calculando imagem 385
calculando imagem 386
calculando imagem 387
calculando imagem 388
calculando imagem 389
calculando imagem 390
calculando imagem 391
calculando imagem 392
calculando imagem 393
calculando imagem 394
calculando imagem 395
calculando imagem 396
calculando imagem 397
calculando imagem 398
calculando imagem 399
calculando imagem 400
calculando imagem 401
calculando imagem 402
calculando imagem 403
calculando imagem 404
calculando imagem 405
calculando imagem 406
calculando imagem 407
calculando imagem 408
calculando imagem 409
calculando imagem 410
calculando imagem 411
calculando imagem 412
calculando imagem 413
calculando imagem 414
calculando imagem 415
calculando imagem 416
calculando imagem 417
calculando imagem 418
calculando imagem 419
calculando imagem 420
calculando imagem 421
calculando imagem 422
calculando imagem 423
calculando

In [ ]:
#data_analise=pd.read_csv("csv/{}_all_methods_analise.csv".format(path.split("/")[-1]))

In [13]:

dataframe.to_csv("720_images_all_methods_analise.csv".format(path.split("/")[-1]))
contas=dataframe.drop(["imagem","Esperado","index_img"], axis=1)
contas

,Hprojection,Hough,Fourier,Mediana,Media,Moda,erro_mediana,erro_media,erro_moda
0,-8,82.0,84.0,82.0,53.0,83.0,89,60,0
1,-86,4.0,3.0,3.0,-26.0,3.5,89,60,89.5
2,-36,54.0,55.0,54.0,24.0,54.5,0,60,0.5
3,-68,-67.0,-69.0,-68.0,-68.0,-68.0,0,0,0
4,-40,-40.0,51.0,-40.0,-10.0,5.5,0,30,45.5
5,-7,-7.0,-6.0,-7.0,-7.0,-6.5,0,0,0.5
6,-36,54.0,55.0,54.0,24.0,54.5,0,60,0.5
7,-35,-35.0,-34.0,-35.0,-35.0,-34.5,0,0,0.5
8,-75,-75.0,-78.0,-75.0,-76.0,-76.5,0,89,88.5
9,-56,34.0,36.0,34.0,5.0,35.0,0,61,1


In [7]:
media=np.round(contas.mean(axis=1))
moda_vector=(contas.mode(axis=1,numeric_only=True)).median(axis=1)
mediana=(contas.median(axis=1))

dataframe["Mediana"]=mediana
dataframe["Media"]=media
dataframe["Moda"]=moda_vector

In [8]:
dataframe["erro_mediana"]=(dataframe["Esperado"]+dataframe["Mediana"])%90
dataframe["erro_media"]=(dataframe["Esperado"]+dataframe["Media"])%90
dataframe["erro_moda"]=(dataframe["Esperado"]+dataframe["Moda"])%90

In [14]:
dataframe

,imagem,Hprojection,Hough,Fourier,Esperado,index_img,Mediana,Media,Moda,erro_mediana,erro_media,erro_moda
0,50646501-6511.page_3..png,-8,82.0,84.0,-83,0,82.0,53.0,83.0,89,60,0
1,50448615-8615.page_0..png,-86,4.0,3.0,-4,1,3.0,-26.0,3.5,89,60,89.5
2,71989648.page_0..png,-36,54.0,55.0,-54,2,54.0,24.0,54.5,0,60,0.5
3,2057444860.page_0..png,-68,-67.0,-69.0,68,3,-68.0,-68.0,-68.0,0,0,0
4,0060200963.page_1..png,-40,-40.0,51.0,-50,4,-40.0,-10.0,5.5,0,30,45.5
5,2040279787.page_0..png,-7,-7.0,-6.0,7,5,-7.0,-7.0,-6.5,0,0,0.5
6,80221250_80221251.page_0..png,-36,54.0,55.0,-54,6,54.0,24.0,54.5,0,60,0.5
7,50117087-7087.page_0..png,-35,-35.0,-34.0,35,7,-35.0,-35.0,-34.5,0,0,0.5
8,527967642+-7643.page_0..png,-75,-75.0,-78.0,75,8,-75.0,-76.0,-76.5,0,89,88.5
9,0000242151.page_0..png,-56,34.0,36.0,56,9,34.0,5.0,35.0,0,61,1


In [10]:
dataframe["erro_mediana"].value_counts()

0.0     417
89.0    131
1.0      76
88.0     53
2.0      28
3.0       4
87.0      3
4.0       3
86.0      1
84.0      1
25.0      1
9.0       1
Name: erro_mediana, dtype: int64

In [22]:
export_csv = dataframe.to_csv ('/Users/thales/Documents/Faculdade/mc040/prototype/csv/720_img_all_methods.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [17]:
dataframe

,imagem,Hprojection,Hough,Fourier,Esperado,index_img,Mediana,Media,Moda,erro_mediana,erro_media,erro_moda
0,50646501-6511.page_3..png,-8,82.0,84.0,-83,0,82.0,53.0,83.0,89,60,0
1,50448615-8615.page_0..png,-86,4.0,3.0,-4,1,3.0,-26.0,3.5,89,60,89.5
2,71989648.page_0..png,-36,54.0,55.0,-54,2,54.0,24.0,54.5,0,60,0.5
3,2057444860.page_0..png,-68,-67.0,-69.0,68,3,-68.0,-68.0,-68.0,0,0,0
4,0060200963.page_1..png,-40,-40.0,51.0,-50,4,-40.0,-10.0,5.5,0,30,45.5
5,2040279787.page_0..png,-7,-7.0,-6.0,7,5,-7.0,-7.0,-6.5,0,0,0.5
6,80221250_80221251.page_0..png,-36,54.0,55.0,-54,6,54.0,24.0,54.5,0,60,0.5
7,50117087-7087.page_0..png,-35,-35.0,-34.0,35,7,-35.0,-35.0,-34.5,0,0,0.5
8,527967642+-7643.page_0..png,-75,-75.0,-78.0,75,8,-75.0,-76.0,-76.5,0,89,88.5
9,0000242151.page_0..png,-56,34.0,36.0,56,9,34.0,5.0,35.0,0,61,1
